In [1]:
import os
import fnmatch
import subprocess
import re
import simpleMenu
import ConfigParser
import logging
from logging.handlers import RotatingFileHandler
import sys

appShortName = 'fileStreamPortfolio'

# # Create Logger
# logger = logging.getLogger(__name__)

# # remove any existing handlers (needed for running within an IDE such as Jupyter
# for each in range (0, len(logger.handlers)):
#     logger.removeHandler(logger.handlers[0])
# datefmt = '%y-%m-%d %H:%M:%S'

# logger.setLevel(logging.DEBUG)
# console_handler = logging.StreamHandler()
# console_handler.setLevel(logging.ERROR)
# consoleformat = '%(asctime)s %(levelname)s - %(message)s'

# logformat = '%(asctime)s %(levelname)s %(module)s - %(funcName)s: %(message)s'

# rotation_handler = RotatingFileHandler(appShortName+'.log', maxBytes = 5000, backupCount = 5)
# rotation_handler.setFormatter(logging.Formatter(fmt = logformat, datefmt = datefmt))

# logger.addHandler(console_handler)
# logger.addHandler(rotation_handler)

# # # setup log handlers
# # logformat = '%(asctime)s %(levelname)s %(module)s - %(funcName)s: %(message)s'
# # streamformat = '%(asctime)s %(levelname)s - %(message)s'
# # datefmt = '%y-%m-%d %H:%M:%S'

# # logging.basicConfig(filename = appShortName+'.log', filemode = 'w',
# #                     format = logformat, datefmt = datefmt)

# # stream_handler = logging.StreamHandler(sys.stderr)
# # stream_handler.setFormatter(logging.Formatter(fmt = streamformat, datefmt = ''))
# # stream_handler.setLevel(logging.ERROR)
# # rotation_handler = RotatingFileHandler(appShortName+'.log', maxBytes = 5000, backupCount = 5)
# # rotation_handler.setFormatter(logging.Formatter(fmt = logformat, datefmt = datefmt))

# # logger = logging.getLogger(__name__)

# # # this removes any stray handlers created prior to this execution - handy when testing within an IDE such as Jupyter
# # if len(logger.handlers) > 0:
# #     for each in range (0, len(logger.handlers)):
# #         logger.removeHandler(logger.handlers[0])

# # # add the stream handler
# # logger.addHandler(stream_handler)
# # logger.addHandler(rotation_handler)

# # logger.setLevel(logging.DEBUG)
# # logger.info('Log Started')
# # logger.info('Log Level: {0}'.format(logger.level))

# # end setup log handlers

In [ ]:
class modLogger(object):
    def __init__(self):
        logger = logging.getLogger(__name__)
        logger

In [ ]:
foo = myProgram()
foo.log()

In [ ]:
logging.critical('whoa')
logging.debug('stuff')

In [ ]:
logging.debug('foo')

In [ ]:
configPath = os.path.expanduser('~/.config/'+appShortName)
configFile = configPath+'/config.ini'
print configFile
config = ConfigParser.ConfigParser()
config.read(configFile)
config.add_section('Main')
try:
    with open (configFile, 'wb') as myFile:
        config.write(myFile)
except IOError:
    try:
        os.makedirs(configPath)
    except Exception as e:
        print e

In [ ]:
configfile = './fileStreamPortfolio.ini'
config = ConfigParser.ConfigParser()
config.read(configfile)

try:
    config.add_section('Main')
    config.add_section('Recent')
except ConfigParser.DuplicateSectionError as e:
    pass

config.set('Main', 'DefaultMountPoint', '/Volumes/GoogleDrive')
config.set('Main', 'TeamDrive', '')
config.set('Main', 'PortfolioPath', '')

config.set('Recent', 'GradeFolder', '')

try:
    myConfig = open(configfile, 'wb')
    config.write(myConfig)
except Exception as e:
    print e

In [ ]:
class configuration(object):
    def __init__(self, configPath = os.path.expanduser('~/.config/'+appShortName)):
        self.configFile = os.path.join(self.configPath, 'config.ini')
        self.getConfig()
        
    def getConfig(self):
        self.parser = ConfigParser.SafeConfigParser()
        
        mainSection = 'Main'
        # key: [method to set, 'default value if not found']
        settings = {'mountPoint': [self.parser.get, '/Volumes/GoogleDrive'],
                    'teamDrive': [self.parser.get, ''],
                    'lastGradeFolders': [self.parser.get, './gradefolders.txt']}
        
        # check if configuation path exists
        if len(self.parser.read(self.configFile)) <= 0:
            try:
                #os.makedirs(os.path.expanduser(self.configPath))
                os.makedirs('/foo/bar')
            except Exception as e:
                print e


In [ ]:
foo = configuration
os.makedirs('/foo/bar/')

In [2]:
class teamDrives(object):
    '''
    make working with google Team Drives through filestream a little easier
    '''
    def __init__(self, mountpoint='/Volumes/GoogleDrive/Team Drives/'):
        self.mountpoint = mountpoint
        self.getDrives()
    
    def getDrives(self):
        self.drives = {}
        try:
            drives = next(os.walk(self.mountpoint))[1]
        except Exception as e:
            logging.critical('error retriving list of Team Drives: {0}'.format(e))
            logging.critical('is Google Drive File Stream application running and configured?')
            logging.critical('mount point: {} is not accessible'.format(self.mountpoint))
            logging.critical('exiting')
            return(0)
        for drive in drives:
            self.drives[drive] = oct(os.stat(self.mountpoint+drive).st_mode & 0o777)
    
    def listrwDrives(self):
        rwDrives = []
        for drive in self.drives:
            if 777 - int(self.drives[drive]) < 277:
                rwDrives.append(drive)
        return(sorted(rwDrives))
    
    def listFolders(self, teamDrive):
        try:
            folders = next(os.walk(self.mountpoint+teamDrive))[1]
        except Exception as e:
            print 'error getting list of Team Drives: {0}'.format(e)
            print 'is Google Drive File Stream application running and configured?'
            raise os.error('Error in os.walk for mount point: {0}'.format(self.mountpoint))
        return(folders)
    
    def find(self, pattern, teamDrive):
        result = []
        for root, dirs, files in os.walk(self.mountpoint+teamDrive):
            for name in dirs:
                if fnmatch.fnmatch(name, pattern):
                    result.append(os.path.join(root, name))
        return result
                
                
    

        

In [3]:
def checkFSMount(mountpoint = '/Volumes/GoogleDrive'):
    logger.debug('Searching for Google Drive File Stream Mount Points')
    # list the local partitions (including FUSE partitions)
    mount = subprocess.check_output(['df', '-hl'])
    mountLines = mount.split('\n')
    mountPoints = []
    
    # makek a list of all the partitions
    for line in mountLines:
        try:
            # re extract anything that looks like a mount point
            mountSearch = re.search('\s+(\/[\S+]{0,})$', line)
            mountPoints.append(mountSearch.group(1))
        # ignore anything that doesn't match the re
        except Exception:
            pass    
    # check for mount point and try to launch the google drive file stream app
    if mountpoint in mountPoints:
        logger.debug('Found mountpoint at: {0}'.format(mountpoint))
        return True
    else:
        logger.info('Google Drive File Stream appears to not be running')
        return False
    


In [11]:
def ltest(i = 'abcdefg'):
    logger = logging.getLogger(__name__)
    logger.debug('stuff: {0}'.format(i))
    logger.info('other stuff')
    logger.critical('important stuff')
    #logging.warn('stuff: {0}'.format(i))

In [12]:
def main():

    # Create the Logger
    logger = logging.getLogger(__name__)

    for each in range (0, len(logger.handlers)):
        logger.removeHandler(logger.handlers[0])

    datefmt = '%y-%m-%d %H:%M:%S'

    logger.setLevel(logging.INFO)

    # file handler
    fileformat = '%(asctime)s %(levelname)s %(module)s - %(funcName)s: %(message)s'
    file_handler = logging.FileHandler(appShortName+'.log')
    file_handler.setLevel(logging.DEBUG)
    file_handler.setFormatter(logging.Formatter(fmt = fileformat, datefmt = datefmt))

    # stream handler
    streamformat = '%(asctime)s %(levelname)s - %(funcName)s: %(message)s'
    stream_handler = logging.StreamHandler(sys.stderr)
    stream_handler.setLevel(logging.CRITICAL)
    stream_handler.setFormatter(logging.Formatter(fmt = streamformat, datefmt = ''))

    # rotation handler
    rotation_handler = RotatingFileHandler(appShortName+'.log', maxBytes = 5000, backupCount = 5)
    rotation_handler.setLevel(logging.DEBUG)
    rotation_handler.setFormatter(logging.Formatter(fmt = fileformat, datefmt = datefmt))


    # add handler to logger
    #logger.addHandler(file_handler)
    logger.addHandler(stream_handler)
    logger.addHandler(rotation_handler)
    
    ltest()
    logger.critical('oh hi')
    
#     if checkFSMount(''):
#         pass
#     else:
#         logger.info('Attempting to start Google Drive File Stream')
#         try:
#             gDriveFS = subprocess.check_call(["open", "-a", "Google Drive File Stream"])
#         except subprocess.CalledProcessError as err:
#             logging.critical('Google Drive File Stream does not appear to be installed. Please download from the link below')            
#             logging.critical('https://support.google.com/drive/answer/7329379?hl=en')
#             logging.critical('exiting')
#             logging.info('OS Error: {0}'.format(err))
#             return(0)
#         if checkFSMount():
#             pass
#         else:
#             print "exiting"
#             return(0)

#     # build a configuration file around this
        
#     # get a list of the avialable team drives
#     myDrives = teamDrives('/foo')
#     try:
#         myDrives.getDrives()
#     except Exception as e:
#         print e
#         return(0)
#     if len(myDrives.listrwDrives()) < 1:
#         print 'No usable Team Drives available; exiting'
#         return(0)
#     else:
#         rwDrivesMenu = simpleMenu.menu(name = 'Team Drives', items = myDrives.listrwDrives())
        
#     myTeamDrive = rwDrivesMenu.loopChoice(optional = True, message = 'Which Team Drive contains the portfolio folder?')

#     folderSearch = raw_input('Please enter part of the portfolio folder name: ')
#     foldersMenu = simpleMenu.menu(name = 'Matching Folders', items = myDrives.find(pattern = '*'+folderSearch+'*', teamDrive = myTeamDrive))
#     myFolder = foldersMenu.loopChoice(optional = True, message = 'Which foldr contains the portfolios?')
    
    
#     # open the student data file and start creating folders as needed

    
main()

2018-04-28 21:24:34,477 CRITICAL - ltest: important stuff
2018-04-28 21:24:34,478 CRITICAL - main: oh hi


##### print logger.level

In [ ]:
class myProgram(object):
    def __init__(self):
        self.appShortName = 'fileStreamPortfolio'
        
        # Create the Logger
        self.logger = logging.getLogger(__name__)
        
        for each in range (0, len(self.logger.handlers)):
            self.logger.removeHandler(self.logger.handlers[0])
        
        datefmt = '%y-%m-%d %H:%M:%S'
        
        self.logger.setLevel(logging.WARNING)
        
        # file handler
        fileformat = '%(asctime)s %(levelname)s %(module)s - %(funcName)s: %(message)s'
        file_handler = logging.FileHandler(self.appShortName+'.log')
        file_handler.setLevel(logging.DEBUG)
        file_handler.setFormatter(logging.Formatter(fmt = fileformat, datefmt = datefmt))
        
        # stream handler
        streamformat = '%(asctime)s %(levelname)s - %(message)s'
        stream_handler = logging.StreamHandler(sys.stderr)
        stream_handler.setLevel(logging.WARN)
        stream_handler.setFormatter(logging.Formatter(fmt = streamformat, datefmt = ''))
        
        # rotation handler
        rotation_handler = RotatingFileHandler(self.appShortName+'.log', maxBytes = 5000, backupCount = 5)
        rotation_handler.setFormatter(logging.Formatter(fmt = streamformat, datefmt = datefmt))
        
        # add handler to logger
        #self.logger.addHandler(file_handler)
        self.logger.addHandler(stream_handler)
        self.logger.addHandler(rotation_handler)
            
        

        
myRun = myProgram()

In [ ]:
def find(pattern, path):
    result = []
    for root, dirs, files in os.walk(path):
        for name in dirs:
            if fnmatch.fnmatch(name, pattern):
                result.append(os.path.join(root, name))
    return result



In [ ]:
find('*Port*', '/Volumes/GoogleDrive/Team Drives/IT Blabla')

In [ ]:
output = [dI for dI in os.listdir('/Volumes/GoogleDrive/Team Drives/IT Blabla') if os.path.isdir(os.path.join('/Volumes/GoogleDrive/Team Drives/IT Blabla',dI))]

print output